# # Análisis de Calidad de Datos: Productos No Encontrados
#
# **Autor:** Franco (franco18min@github.com)
# **Fecha:** 2025-07-18
#
# ## Objetivo
# Este notebook tiene como objetivo analizar los productos que no pudieron ser categorizados durante la ejecución del pipeline de la Capa de Oro en Databricks.
#
# El flujo de trabajo es el siguiente:
# 1. Conectarse a Databricks y descargar la tabla de reporte de productos no encontrados.
# 2. Analizar los datos con `pandas` para identificar patrones.
# 3. Generar una lista de acción para corregir los datos en la fuente de la verdad (`Category.xlsx`).


In [2]:
# --- 1. Configuración e Importaciones ---
# Se importa pandas para el análisis y sys/pathlib para poder importar nuestros módulos locales.
import pandas as pd
import sys
from pathlib import Path
import os

# Añadimos la carpeta 'src' al path de Python para poder importar nuestros módulos
# Esto nos permite reutilizar el conector que ya hemos creado.
project_root = Path.cwd().parent.parent.parent # Navega 3 niveles arriba desde notebooks/jupyters/data_quality_analysis
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

# Ahora podemos importar nuestra función para leer datos desde Databricks
try:
    from tecno_etl.utils.databricks_connector import get_df_from_databricks
    print("Módulo 'databricks_connector' importado exitosamente.")
except ImportError as e:
    print(f"Error al importar el módulo: {e}")
    print("Asegúrate de que la estructura de carpetas es correcta y que el notebook se encuentra en la ubicación esperada.")


Módulo 'databricks_connector' importado exitosamente.


# ## 2. Extracción de Datos Problemáticos
#
# Nos conectamos a la tabla de reporte en Databricks que fue generada por el pipeline de la Capa de Oro. Esta tabla contiene la lista de `codigo_producto` que no encontraron una coincidencia en la tabla de dimensiones `category`.

In [3]:
# --- Definición de la tabla de reporte ---
# El nombre de la tabla se deriva del nombre del archivo original.
# Asegúrate de que coincida con el que se generó en el notebook de la Capa de Oro.
REPORTING_CATALOG = "workspace"
REPORTING_SCHEMA = "tecnomundo_data_reporting"
REPORTING_TABLE = "productos_sin_categoria"

# --- Descarga de datos ---
print(f"Descargando datos de la tabla de reporte: {REPORTING_CATALOG}.{REPORTING_SCHEMA}.{REPORTING_TABLE}")

df_unmatched = get_df_from_databricks(
    catalog=REPORTING_CATALOG,
    schema=REPORTING_SCHEMA,
    table_name=REPORTING_TABLE
)

if df_unmatched is not None:
    print(f"\n¡Éxito! Se descargaron {len(df_unmatched)} códigos de producto no encontrados.")
else:
    print("\nNo se pudieron descargar los datos o la tabla está vacía.")


2025-07-18 18:22:26,532 - INFO - Intentando obtener la tabla 'workspace.tecnomundo_data_reporting.productos_sin_categoria' de Databricks...


Descargando datos de la tabla de reporte: workspace.tecnomundo_data_reporting.productos_sin_categoria


2025-07-18 18:22:28,437 - INFO - ✅ ¡Conexión exitosa a Databricks!
2025-07-18 18:22:28,437 - INFO - Ejecutando consulta: SELECT * FROM `workspace`.`tecnomundo_data_reporting`.`productos_sin_categoria`
C:\Users\tc\Desktop\TecnoMundo_ETL_pipeline\src\tecno_etl\utils\databricks_connector.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
2025-07-18 18:22:52,506 - INFO - ✅ Tabla 'productos_sin_categoria' cargada exitosamente con 7 filas.



¡Éxito! Se descargaron 7 códigos de producto no encontrados.


# ## 3. Análisis Exploratorio
#
# Ahora que tenemos los datos en un DataFrame de `pandas`, podemos analizarlos para entender mejor el problema.

In [4]:
if df_unmatched is not None and not df_unmatched.empty:
    # Mostramos los primeros 10 productos no encontrados
    print("--- Primeros 10 productos no encontrados ---")
    print(df_unmatched.head(10))

    # Contamos cuántos productos únicos no se encontraron
    unique_unmatched_count = df_unmatched['codigo_producto'].nunique()
    print(f"\nNúmero total de códigos de producto únicos no encontrados: {unique_unmatched_count}")

else:
    print("No hay productos no encontrados para analizar. ¡Excelente trabajo!")

--- Primeros 10 productos no encontrados ---
  codigo_producto
0           C7912
1         RC91102
2          RC9199
3         RC85607
4            F434
5         MO24127
6         PC12413

Número total de códigos de producto únicos no encontrados: 7


# ## 4. Conclusiones y Pasos a Seguir
#
# El análisis anterior nos proporciona una lista clara de los códigos de producto que faltan en nuestra tabla de verdad.
#
# ### Lista de Acción:
# 1.  **Revisar la lista de códigos de producto** impresa en la celda anterior.
# 2.  **Investigar cada código:**
#     * ¿Es un producto nuevo que necesita ser añadido?
#     * ¿Es un error de tipeo en el sistema de origen?
# 3.  **Actualizar la Fuente de la Verdad:** Se ha proporcionado un archivo excel con los productos faltantes los cuales seran subidos a databricks para poder actualzar la tabla de dimensiones.
# 4.  **Actualizar la Tabla de Dimensiones:** Ejecutar el pipeline local `run_local_ingestion.py` para recargar la tabla `category` en Databricks con los nuevos productos.
# 5.  **Reprocesar la Capa de Oro:** Volver a ejecutar el notebook `3_1_categorize_sales.ipynb` en Databricks. En esta nueva ejecución, estos productos ahora serán categorizados correctamente.